# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try: 
    # Connect to local Apache Cassandra instance
    cluster = Cluster(['127.0.0.1'])
    # Set session to connect andexecute queries.
    session = cluster.connect()
except Exception as e:
    print(e)



#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Create Table - Insert Data - Query For Question 1

In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# Create table song_and_length use for question 1
# column: 
# - session_id int
# - item_in_session int
# - artist text
# - song text
# - length float
# To uniquely identify each row and allow efficient distribution in Cassandra cluster, 
# session_id and item_in_session columns: are used as table's Primary Key (composite Partition Key). 
query = "CREATE TABLE IF NOT EXISTS song_and_length "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, \
                    PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
# Insert data song_and_length
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
         # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_and_length (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Assign column elements in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
# select song_and_length return  artist, song and length from song_and_length table.
query = "SELECT artist, song, length \
                FROM song_and_length \
                WHERE   session_id = 338 AND \
                        item_in_session = 4"

try:
    songs = session.execute(query)
except Exception as e:
    print(e)

for row in songs:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Create Table - Insert Data - Query For Question 2

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# Create table name_song use for query 2
# Column:
# - user_id int
# - session_id int
# - artist text
# - song text
# - item_in_session int
# - first_name text
# - last_name text
# To uniquely identify each row and allow efficient distribution in Cassandra cluster, 
# user_id and session_id columns: are used as Composite Partition Key in table's Primary Key.
# item_in_session column: is used as Clustering Key in table's Primary Key and allows sorting order of the data.

query = "CREATE TABLE IF NOT EXISTS name_song "
query = query + "(  user_id int, \
                    session_id int, \
                    artist text, \
                    song text, \
                    item_in_session int, \
                    first_name text, \
                    last_name text, \
                    PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

                    

In [12]:
# Insert data name_song
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO name_song ( \
                                        user_id, \
                                        session_id, \
                                        artist, \
                                        song, \
                                        item_in_session, \
                                        first_name, \
                                        last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Insert values
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

In [13]:
# select song_and_length return  artist, song, first_name, and last_name (of user) from name_song table.
query = "SELECT artist, song, first_name, last_name \
                    FROM name_song \
                    WHERE   user_id = 10 AND \
                    session_id = 182 \
                    order by item_in_session" 
                    
                    
try:
    artists = session.execute(query)
except Exception as e:
    print(e)

for row in artists:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Create Table - Insert Data - Query For Question 3

In [14]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Create table first_and_lastname_user use for query 3
# column:
# - song text
# - user_id int
# - first_name text
# - last_name text
# To uniquely identify each row and allow efficient distribution in Cassandra cluster, 
# song, user_id columns: are used as Composite Partition Key in table's Primary Key.
query = "CREATE TABLE IF NOT EXISTS first_and_lastname_user "
query = query + "(  song text, \
                    user_id int, \
                    first_name text, \
                    last_name text, \
                    PRIMARY KEY(song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [15]:
# INSERT data first_and_lastname_user
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO first_and_lastname_user ( \
            song, \
            user_id, \
            first_name, \
            last_name \
        )"

        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
# select first_and_lastname_user return first_name and last_name (of users) from first_and_lastname_user table.
query = "SELECT first_name, last_name \
                    FROM first_and_lastname_user \
                    WHERE song = 'All Hands Against His Own'"
try:
    users = session.execute(query)
except Exception as e:
    print(e)

for row in users:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:

query = "DROP TABLE song_and_length"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query2 = "DROP TABLE name_song"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "DROP TABLE first_and_lastname_user"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()